In [5]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

import numpy as np
import torch
import torch.nn as nn
import torch_cluster # pip install torch-cluster -f https://data.pyg.org/whl/torch-2.1.0+cu121.html
import pandas as pd
from src.data_processing.eda_utils import show_3d_plot_shapenet
import os

np.random.seed(42)
torch.manual_seed(42)    


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
data_path = "../../data/shapenetcore_partanno_segmentation_benchmark_v0_normal/"
airplane_folder = "02691156"
airplane_data = "1a04e3eab45ca15dd86060f189eb133.txt"
airplane_df = pd.read_csv(os.path.join(data_path,airplane_folder,airplane_data), 
                          delimiter=" ", 
                          names = ["x", "y", "z", "r", "g", "b", "label"], 
                          header=None)
classes_code_df = pd.read_csv(os.path.join(data_path,"synsetoffset2category.txt"), 
                          delimiter="\t", 
                          names = ["class", "code"], 
                          header=None)
classes_code_df['code'] = "0" + classes_code_df['code'].astype(str)

In [7]:
show_3d_plot_shapenet(airplane_df)

In [39]:
points = torch.tensor(airplane_df.values[:,:3], dtype=torch.float32)
points_features = torch.tensor(airplane_df.values[:,3:6], dtype=torch.float32)

In [69]:
points.shape

torch.Size([2816, 3])

In [47]:
clusters = torch_cluster.grid.grid_cluster(points, torch.tensor([0.1,0.1,0.1]))

In [62]:
torch.unique(clusters)

tensor([  9,  10,  11,  24,  25,  26,  37,  38,  39,  45,  46,  52,  53,  54,
         59,  60,  65,  66,  67,  68,  70,  73,  74,  75,  80,  81,  82,  87,
         91,  92,  93,  94,  95,  96,  98,  99, 100, 101, 102, 103, 104, 108,
        109, 110, 115, 116, 121, 122, 123, 124, 126, 129, 130, 131, 136, 137,
        138, 149, 150, 151, 157, 164, 165, 166, 177, 178, 179, 185, 186, 192,
        193, 194, 205, 206, 207, 220, 221])

In [63]:
cluster_max_features = torch.max(points_features[torch.argwhere(clusters == 193).squeeze(1)], dim = 0)[0]
cluster_max_features

tensor([0.8060, 1.0000, 0.9610])

In [68]:
torch.cat((clusters.reshape(-1,1), points_features), dim=1)

tensor([[ 5.2000e+01,  6.9870e-03,  9.9960e-01,  2.7960e-02],
        [ 1.9300e+02, -3.4790e-02, -9.9920e-01,  1.7640e-02],
        [ 1.9400e+02,  4.6400e-04, -1.0000e+00,  5.9000e-04],
        ...,
        [ 1.6500e+02,  2.7930e-02,  9.9950e-01, -1.3960e-02],
        [ 9.8000e+01,  3.1400e-02,  9.9580e-01,  8.6250e-02],
        [ 1.6400e+02,  0.0000e+00,  1.0000e+00,  0.0000e+00]])